In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train=torch.load('/kaggle/input/gesture/train.pt',weights_only=True) 
test=torch.load('/kaggle/input/gesture/test.pt',weights_only=True)  
val=torch.load('/kaggle/input/gesture/val.pt',weights_only=True) 

In [ ]:

print("train:",train) 
print("test:",test)
print("val:",val)

In [ ]:
print(train.keys())
print(test.keys())
print(val.keys())

In [ ]:
# Extract the data and labels
train_samples, train_labels = train['samples'], train['labels']
val_samples, val_labels = val['samples'], val['labels']
test_samples, test_labels = test['samples'], test['labels'] 

print("train:",train_samples.shape,train_labels.shape) 
print("val:",val_samples.shape,val_labels.shape)
print("test:",test_samples.shape,test_labels.shape)

In [ ]:
print(train_samples[0])  
print(train_labels[0]) 

In [ ]:
train_samples = train_samples.float()  # Convert to float32
val_samples = val_samples.float()      # Convert to float32
test_samples = test_samples.float()    # Convert to float32

In [ ]:
print(train_samples.dtype)

In [ ]:
# Normalize and convert to float32
train_samples = train_samples / train_samples.max()
val_samples = val_samples / val_samples.max()
test_samples = test_samples / test_samples.max()

# Calculate mean and std using train_samples
train_mean = train_samples.mean(dim=[0, 2], keepdim=True)  # Mean for each channel (over batch and time)
train_std = train_samples.std(dim=[0, 2], keepdim=True)    # Std for each channel (over batch and time)

# Standardize the train_samples, val_samples, and test_samples using train_mean and train_std
train_samples = (train_samples - train_mean) / train_std
val_samples = (val_samples - train_mean) / train_std  # Use train mean and std for val_samples
test_samples = (test_samples - train_mean) / train_std  # Use train mean and std for test_samples

# Convert to float32
# train_samples = train_samples.float()
# val_samples = val_samples.float()
# test_samples = test_samples.float()

# Check the shapes and types
print(train_samples.shape, train_samples.dtype)
print(val_samples.shape, val_samples.dtype)
print(test_samples.shape, test_samples.dtype) 

print("Train mean:", train_mean)
print("Train std:", train_std)


In [ ]:
# Function to check for NaN values in the tensor
def check_for_missing_values(data_tensor, dataset_name="Dataset"):
    # Check for NaN values
    nan_values = torch.isnan(data_tensor)

    # Check if there are any NaN values
    if nan_values.any():
        print(f"{dataset_name} has missing (NaN) values.")
    else:
        print(f"{dataset_name} has no missing values.")

    # Optionally, print the indices where NaNs are located (can be large if the tensor is big)
    nan_indices = torch.nonzero(nan_values)
    if nan_indices.numel() > 0:
        print(f"Indices of NaN values in {dataset_name}:")
        print(nan_indices)

# Check for missing values in train, validation, and test datasets
check_for_missing_values(train_samples, "Train Samples")
check_for_missing_values(val_samples, "Validation Samples")
check_for_missing_values(test_samples, "Test Samples")

In [ ]:
# Reshape to (batch_size, sequence_length, input_size) 
train_samples = train_samples.permute(0, 2, 1).to(device)
val_samples = val_samples.permute(0, 2, 1).to(device)
test_samples = test_samples.permute(0, 2, 1).to(device) 
print(train_samples.shape) 
print(val_samples.shape)
print(test_samples.shape)

In [ ]:
# Assuming you have a labels tensor (train_labels, val_labels, or test_labels)
unique_labels = torch.unique(train_labels)  # You can also use val_labels or test_labels
print(f"Unique labels: {unique_labels}")
num_classes = len(unique_labels)
print(f"Number of classes: {num_classes}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import GradScaler, autocast

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Self-Supervised Learning Encoder Model
class ContrastiveEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ContrastiveEncoder, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=2, batch_first=True, bidirectional=True)

    def forward(self, x):
        out, _ = self.lstm(x)
        return out[:, -1, :]  # Last hidden state as representation

# Contrastive Loss for Self-Supervised Learning
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)
        loss = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                          (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss

# Initialize the encoder
input_size = 3
hidden_size = 128
encoder = ContrastiveEncoder(input_size=input_size, hidden_size=hidden_size).to(device)
contrastive_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
contrastive_criterion = ContrastiveLoss()

# Contrastive Learning Dataset Preparation (Self-Supervised)
def create_pairs(data, labels):
    pairs, pair_labels = [], []
    for i in range(len(data)):
        for j in range(len(data)):
            if i != j:
                pairs.append((data[i], data[j]))
                pair_labels.append(1 if labels[i] == labels[j] else 0)
    return torch.stack([p[0] for p in pairs]), torch.stack([p[1] for p in pairs]), torch.tensor(pair_labels).to(device)

# Create pairs and prepare DataLoader
train_sample1, train_sample2, train_pair_labels = create_pairs(train_samples, train_labels)

# Contrastive learning data loader
batch_size = 32
train_dataset = TensorDataset(train_sample1, train_sample2, train_pair_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Train the encoder with contrastive loss and AMP
scaler = GradScaler()  # For automatic mixed precision
def train_encoder(encoder, data_loader, criterion, optimizer, num_epochs=10):
    encoder.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for sample1, sample2, labels in data_loader:
            sample1, sample2, labels = sample1.to(device), sample2.to(device), labels.float().to(device)
            optimizer.zero_grad()
            with autocast():  # Use AMP for faster training
                output1, output2 = encoder(sample1), encoder(sample2)
                loss = criterion(output1, output2, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(data_loader):.4f}")

train_encoder(encoder, train_loader, contrastive_criterion, contrastive_optimizer, num_epochs=10)

# Classification Model using Learned Representations
class GestureClassifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super(GestureClassifier, self).__init__()
        self.encoder = encoder
        self.fc = nn.Linear(2 * hidden_size, num_classes)

    def forward(self, x):
        with torch.no_grad():  # Freeze encoder for classification task
            x = self.encoder(x)
        return self.fc(x)

num_classes = 8
classifier = GestureClassifier(encoder, num_classes).to(device)
classification_optimizer = optim.Adam(classifier.parameters(), lr=0.001)
classification_criterion = nn.CrossEntropyLoss()

# Classification DataLoader
train_classification_dataset = TensorDataset(train_samples, train_labels)
train_classification_loader = DataLoader(train_classification_dataset, batch_size=batch_size, shuffle=True)
val_classification_dataset = TensorDataset(val_samples, val_labels)
val_classification_loader = DataLoader(val_classification_dataset, batch_size=batch_size)

# Training Function for Classification Model with Early Stopping
def train_classifier(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=3):
    model.train()
    best_accuracy = 0
    patience_counter = 0
    
    for epoch in range(num_epochs):
        for samples, labels in train_loader:
            samples, labels = samples.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(samples)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation with Early Stopping
        val_accuracy = evaluate_model(model, val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')
        
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            patience_counter = 0  # Reset patience counter if accuracy improves
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print("Early stopping due to no improvement.")
            break

# Evaluation function for the classification model
def evaluate_model(model, data_loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for samples, labels in data_loader:
            samples, labels = samples.to(device), labels.to(device)
            outputs = model(samples)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train and Evaluate the Classifier
train_classifier(classifier, train_classification_loader, val_classification_loader, classification_criterion, classification_optimizer, num_epochs=10)

# Final Test Accuracy
test_classification_dataset = TensorDataset(test_samples, test_labels)
test_classification_loader = DataLoader(test_classification_dataset, batch_size=batch_size)
test_accuracy = evaluate_model(classifier, test_classification_loader)
print(f'Test Accuracy: {test_accuracy:.2f}%')
